In [7]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import librosa 

Ler os arquivos de áudio para coletar os coeficientes e os os labels. Os labels serão divididos da seguinte maneira:
- Anger: 0
- Disgust: 1
- Fear: 2
- Happy: 3
- Neutral: 4
- Sad: 5

In [8]:
base_dir = 'Dados'

In [2]:
data, sampling_rate = librosa.load('C:/Users/yfrom/OneDrive/Área de Trabalho/UFRJ/10/Proc voz/Dados/anger.wav')

Vou ler apenas 150 audios de cada emoção pq eu não quero queimar meu computador

In [13]:
label_map = {'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5}
lst = []

for emotion in label_map.keys():
    folder_path = os.path.join(base_dir, emotion)
    audios_number = 0
    # Iterate over the audio files in the folder
    for audio_file in os.listdir(folder_path):
        # print(os.path.join(folder_path, audio_file))
        if audios_number <= 150:
            X, sample_rate = librosa.load(os.path.join(folder_path, audio_file))
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)         
            file = label_map[emotion]
            arr = mfccs, file
            lst.append(arr)        
            audios_number += 1
        else: break
    print(audios_number, emotion, 'audio files loaded')

151 anger audio files loaded
151 disgust audio files loaded
151 fear audio files loaded
151 happy audio files loaded
151 neutral audio files loaded
151 sad audio files loaded


In [14]:
X, y = zip(*lst)

In [15]:
X = np.asarray(X)
y = np.asarray(y)


X.shape, y.shape

((906, 40), (906,))

In [16]:
import joblib

X_name = 'X.joblib'
y_name = 'y.joblib'
save_dir = 'Dados'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [17]:
X = joblib.load('Dados/X.joblib')
y = joblib.load('Dados/y.joblib')

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Neural Network

In [42]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [43]:
x_traincnn.shape, x_testcnn.shape

((607, 40, 1), (299, 40, 1))

In [58]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
model = Sequential()

model.add(Conv1D(128, 5,padding='same',  input_shape=(40,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))
opt = optimizers.RMSprop(learning_rate=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [59]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 40, 128)        │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_18 (Activation)      │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 6, 128)         │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_19 (Activation)      │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 6)              │         4,614 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_20 (Activation)      │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 87,430 (341.52 KB)

 Trainable params: 87,430 (341.52 KB)

 Non-trainable params: 0 (0.00 B)

In [60]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [66]:
np.sum(y_test == None)

0

In [61]:
print("x_traincnn shape:", x_traincnn.shape)
print("y_train shape:", y_train.shape)
print("x_testcnn shape:", x_testcnn.shape)
print("y_test shape:", y_test.shape)

x_traincnn shape: (607, 40, 1)
y_train shape: (607,)
x_testcnn shape: (299, 40, 1)
y_test shape: (299,)


In [62]:
cnnhistory = model.fit(x_traincnn, y_train, batch_size=16, epochs=10, validation_data=(x_testcnn, y_test))

Epoch 1/10


ValueError: None values not supported.

In [73]:
def verificar_dados(x_traincnn, y_train, x_testcnn, y_test):
    # Verifica se as variáveis estão definidas
    assert x_traincnn is not None, "x_traincnn contém valores None"
    assert y_train is not None, "y_train contém valores None"
    assert x_testcnn is not None, "x_testcnn contém valores None"
    assert y_test is not None, "y_test contém valores None"
    
    # Verifica se os tipos de dados estão corretos
    assert isinstance(x_traincnn, np.ndarray), f"x_traincnn não é um ndarray, mas {type(x_traincnn)}"
    assert isinstance(y_train, np.ndarray), f"y_train não é um ndarray, mas {type(y_train)}"
    assert isinstance(x_testcnn, np.ndarray), f"x_testcnn não é um ndarray, mas {type(x_testcnn)}"
    assert isinstance(y_test, np.ndarray), f"y_test não é um ndarray, mas {type(y_test)}"
    
    # Verifica as formas dos arrays
    print("Forma de x_traincnn:", x_traincnn.shape)
    print("Forma de y_train:", y_train.shape)
    print("Forma de x_testcnn:", x_testcnn.shape)
    print("Forma de y_test:", y_test.shape)
    
    # Verifica as dimensões
    assert x_traincnn.ndim == 3, f"x_traincnn deve ter 3 dimensões, mas tem {x_traincnn.ndim}"
    assert x_testcnn.ndim == 3, f"x_testcnn deve ter 3 dimensões, mas tem {x_testcnn.ndim}"
    assert y_train.ndim == 1, f"y_train deve ter 1 dimensão, mas tem {y_train.ndim}"
    assert y_test.ndim == 1, f"y_test deve ter 1 dimensão, mas tem {y_test.ndim}"
    
    # Verifica os valores únicos em y_train e y_test
    print("Valores únicos em y_train:", np.unique(y_train))
    print("Valores únicos em y_test:", np.unique(y_test))
    
    # Verifica se há valores nulos
    assert not np.any(np.isnan(x_traincnn)), "x_traincnn contém valores NaN"
    assert not np.any(np.isnan(y_train)), "y_train contém valores NaN"
    assert not np.any(np.isnan(x_testcnn)), "x_testcnn contém valores NaN"
    assert not np.any(np.isnan(y_test)), "y_test contém valores NaN"
    
    print("Todas as verificações passaram com sucesso.")

# Execute a função de verificação antes do treinamento
verificar_dados(x_traincnn, y_train, x_testcnn, y_test)

Forma de x_traincnn: (607, 40, 1)
Forma de y_train: (607,)
Forma de x_testcnn: (299, 40, 1)
Forma de y_test: (299,)
Valores únicos em y_train: [0 1 2 3 4 5]
Valores únicos em y_test: [0 1 2 3 4 5]
Todas as verificações passaram com sucesso.


In [75]:
cnnhistory = model.fit(x_traincnn, y_train, batch_size=16, epochs=10, validation_data=(x_testcnn, y_test))

Epoch 1/10


AttributeError: 'NoneType' object has no attribute 'items'

In [79]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=3)

# Tentativa de treinamento sem validation_data
try:
    cnnhistory = model.fit(x_traincnn, y_train, batch_size=16, epochs=10, callbacks=[early_stopping])
except Exception as e:
    print("Erro durante o treinamento:", str(e))

# Treinamento com validation_data para comparar
try:
    cnnhistory = model.fit(x_traincnn, y_train, batch_size=16, epochs=10, validation_data=(x_testcnn, y_test), callbacks=[early_stopping])
except Exception as e:
    print("Erro durante o treinamento com validation_data:", str(e))

Epoch 1/10
Erro durante o treinamento: 'NoneType' object has no attribute 'items'
Epoch 1/10
Erro durante o treinamento com validation_data: 'NoneType' object has no attribute 'items'
